# Chap05 变形

In [1]:
import numpy as np
import pandas as pd

## 长宽表的变形
1. 长表：信息比较详细的表
2. 宽表：信息比较汇总的表
3. 信息等价，只是数值的呈现方式不同
### `pivot`长表变宽表的函数
1. 三要素：变形后的行索引`index`、需要转到列索引的列`columns`、列和行索引对应的数值`values`
2. 新生成表的列索引是**columns**对应列的唯一值，新表的行索引是**index**对应列的唯一值，**values**对应了想要展示的数值列
3. pivot进行变形操作需要满足唯一性的要求，即新表中的行索引对应的列和列索引对应的列的组合必须唯一
4. 三个参数允许被设置为列表
### `pivot_table`
1. 如果不满足唯一性条件，必须通过聚合操作使得相同行列组合对应的多个值变为一个值

In [2]:
# 长表
pd.DataFrame({'Gender':['F','F','M','M'], 'Height':[163,160,175,180]})

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [3]:
# 宽表
pd.DataFrame({'Height:F':[163,160], 'Height:M':[175,180]})

,Height:F,Height:M
0,163,175
1,160,180


In [4]:
df = pd.DataFrame({'Class': [1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [8]:
df.pivot(index=['Class','Name'], columns='Subject', values='Grade')

,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,75
2,Si Li,90,85


In [ ]:
df = pd.DataFrame({'Name':['San Zhang','San Zhang','San Zhang']})